# 🏭 工場生産実績データで学ぶPandas集中講座

## 📋 講座概要
- **目標**: Pandasを使った工場データの分析
- **内容**: データ読み込み・探索・可視化・分析
- **時間**: 約2時間

## 1. 環境準備とライブラリインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# 設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

print("📊 Pandas学習環境の準備完了！")
print(f"Pandas バージョン: {pd.__version__}")
print(f"NumPy バージョン: {np.__version__}")

## 2. 工場生産実績データの生成

In [ ]:
def create_factory_data():
    """工場生産実績データを生成"""
    np.random.seed(42)  # 再現性のため
    
    # 日付範囲（過去30日）
    dates = pd.date_range(
        start=datetime.now() - timedelta(days=30),
        end=datetime.now(),
        freq='D'
    )
    
    # 工場設定
    product_lines = ['ラインA', 'ラインB', 'ラインC', 'ラインD']
    shifts = ['朝番', '昼番', '夜番']
    
    data = []
    
    for date in dates:
        for line in product_lines:
            for shift in shifts:
                # 基本生産量（ラインごと）
                base_production = {
                    'ラインA': 100, 'ラインB': 120, 
                    'ラインC': 80, 'ラインD': 90
                }[line]
                
                # 曜日・シフト調整
                weekday_factor = 0.7 if date.weekday() >= 5 else 1.0
                shift_factor = {'朝番': 1.0, '昼番': 1.1, '夜番': 0.9}[shift]
                
                # 生産数計算
                production = int(base_production * weekday_factor * shift_factor * 
                               np.random.normal(1.0, 0.15))
                production = max(0, production)
                
                # その他指標
                defects = int(production * np.random.uniform(0.05, 0.15))
                good_products = production - defects
                operating_hours = np.random.uniform(7.0, 8.0)
                productivity = good_products / operating_hours if operating_hours > 0 else 0
                quality_rate = (good_products / production * 100) if production > 0 else 0
                
                data.append({
                    '日付': date.strftime('%Y-%m-%d'),
                    '製品ライン': line,
                    'シフト': shift,
                    '生産数': production,
                    '不良品数': defects,
                    '良品数': good_products,
                    '稼働時間': round(operating_hours, 1),
                    '作業者数': np.random.choice([3, 4, 5]),
                    'エラー回数': np.random.poisson(2),
                    '設備効率': round(np.random.uniform(0.75, 0.95), 3),
                    '生産性': round(productivity, 1),
                    '品質率': round(quality_rate, 1)
                })
    
    return pd.DataFrame(data)

# データ生成
factory_df = create_factory_data()
factory_df.to_csv('factory_production.csv', index=False, encoding='utf-8-sig')

print(f"📈 工場データ生成完了: {len(factory_df)} レコード")
print(f"📅 期間: {factory_df['日付'].min()} ～ {factory_df['日付'].max()}")
print(f"🏭 製品ライン: {factory_df['製品ライン'].nunique()} ライン")
print(f"⏰ シフト: {factory_df['シフト'].nunique()} シフト")

# データの最初の5行を表示
factory_df.head()

## 3. データの読み込みと基本情報確認

In [ ]:
# CSVファイルの読み込み
df = pd.read_csv('factory_production.csv')
df['日付'] = pd.to_datetime(df['日付'])

print("📂 データ読み込み完了")
print(f"📐 データ形状: {df.shape} (行数, 列数)")
print(f"📊 メモリ使用量: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")

# データフレームの情報表示
print("\n📋 データフレーム情報:")
df.info()

In [ ]:
print("📈 数値データの基本統計量:")
df.describe()

## 4. データ探索と可視化

In [ ]:
# 製品ライン別サマリー
line_summary = df.groupby('製品ライン').agg({
    '生産数': ['mean', 'sum'],
    '品質率': 'mean',
    '生産性': 'mean'
}).round(1)

# カラム名を平坦化
line_summary.columns = ['平均生産数', '総生産数', '平均品質率', '平均生産性']

print("🏭 製品ライン別サマリー:")
line_summary

In [ ]:
# 製品ライン別の可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 生産数の分布
axes[0,0].hist(df['生産数'], bins=20, edgecolor='black', alpha=0.7, color='skyblue')
axes[0,0].set_title('生産数の分布')
axes[0,0].set_xlabel('生産数')
axes[0,0].set_ylabel('頻度')

# 2. 製品ライン別平均生産数
line_summary['平均生産数'].plot(kind='bar', ax=axes[0,1], color='lightgreen')
axes[0,1].set_title('製品ライン別平均生産数')
axes[0,1].set_ylabel('生産数')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. シフト別生産数の箱ひげ図
df.boxplot(column='生産数', by='シフト', ax=axes[1,0])
axes[1,0].set_title('シフト別生産数分布')
axes[1,0].set_xlabel('シフト')

# 4. 生産数と品質率の散布図
axes[1,1].scatter(df['生産数'], df['品質率'], alpha=0.6, color='coral')
axes[1,1].set_xlabel('生産数')
axes[1,1].set_ylabel('品質率 (%)')
axes[1,1].set_title('生産数と品質率の関係')

plt.tight_layout()
plt.show()

print(f"📊 生産数の統計:")
print(f"   平均: {df['生産数'].mean():.1f}")
print(f"   中央値: {df['生産数'].median():.1f}")
print(f"   標準偏差: {df['生産数'].std():.1f}")

## 5. データ分析実践

In [ ]:
print("📊 パフォーマンス分析:")

# 高生産データの分析
high_production = df[df['生産数'] > df['生産数'].mean()]
print(f"\n🎯 高生産日（平均超）: {len(high_production)} / {len(df)} レコード")
print(f"📊 割合: {len(high_production) / len(df) * 100:.1f}%")
print(f"🏆 高生産日の平均品質率: {high_production['品質率'].mean():.1f}%")

# 最高パフォーマンスの特定
best_performance = df.loc[df['生産性'].idxmax()]
print(f"\n⚡ 最高生産性記録:")
print(f"   生産性: {best_performance['生産性']} 個/時間")
print(f"   日付: {best_performance['日付'].date()}")
print(f"   ライン: {best_performance['製品ライン']}")
print(f"   シフト: {best_performance['シフト']}")

# シフト別平均値
shift_avg = df.groupby('シフト')[['生産数', '品質率', '生産性']].mean().round(1)
print(f"\n⏰ シフト別平均値:")
print(shift_avg)

In [ ]:
# 時系列トレンド分析
daily_trend = df.groupby('日付').agg({
    '生産数': 'sum',
    '品質率': 'mean',
    '生産性': 'mean'
}).round(1)

# トレンドの可視化
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# 日別総生産数
daily_trend['生産数'].plot(ax=axes[0], color='blue', linewidth=2)
axes[0].set_title('日別総生産数の推移')
axes[0].set_ylabel('総生産数')
axes[0].grid(True, alpha=0.3)

# 日別平均品質率
daily_trend['品質率'].plot(ax=axes[1], color='green', linewidth=2)
axes[1].set_title('日別平均品質率の推移')
axes[1].set_ylabel('品質率 (%)')
axes[1].grid(True, alpha=0.3)

# 日別平均生産性
daily_trend['生産性'].plot(ax=axes[2], color='red', linewidth=2)
axes[2].set_title('日別平均生産性の推移')
axes[2].set_ylabel('生産性 (個/時間)')
axes[2].set_xlabel('日付')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📈 最近5日間のトレンド:")
print(daily_trend.tail())

## 6. まとめ

In [ ]:
print("📋 分析結果サマリー:")
print("="*50)

# 総合統計
total_production = df['生産数'].sum()
avg_quality = df['品質率'].mean()
avg_productivity = df['生産性'].mean()
best_line = line_summary['平均生産数'].idxmax()
best_shift = shift_avg['生産数'].idxmax()

print(f"📊 期間総生産数: {total_production:,} 個")
print(f"📈 平均品質率: {avg_quality:.1f}%")
print(f"⚡ 平均生産性: {avg_productivity:.1f} 個/時間")
print(f"🏆 最高パフォーマンスライン: {best_line}")
print(f"⭐ 最高パフォーマンスシフト: {best_shift}")

print(f"\n💾 データファイル 'factory_production.csv' が保存されました。")
print(f"✅ Pandas集中講座完了！")